# **Detecting outliers**

This script helps to detect mild and extreme outliers of all numerical variables in a dataset, maybe derived from mistakes in data input or some other reason.  

The output is a txt with:

>Column name 
>>Extreme outliers at rows: 
>>>Index of rows with extreme outliers (and the outliers values)  
>>>Index of rows with mild outliers (and the outliers values) 

>>Outliers at rows:  
>>>187	(value: 15.9)
>>>240	(value: 15.9)
>>>30	(value: 16.3)
>>>88	(value: 17.0)
>>>190	(value: 20.0)

Example:

>Column Yield:  
>>Extreme outliers at rows:  
>>>190	(value: 20.0)  

>>Outliers at rows:  
>>>187	(value: 15.9)  
>>>240	(value: 15.9)  
>>>30	(value: 16.3)  
>>>88	(value: 17.0)  
>>>190	(value: 20.0)  

First, we import all required libraries and packages:

In [3]:
import pandas as pd
import numpy as np

Now we load our data file. It it is a .TXT file we'll use:

In [4]:
filename='data/data.txt'
data = pd.read_table(filename)
data

,Variety,Vigour,NFruits,Seeds,SepalLength,PetalLength,Brix_Day1,Brix_Day2,Brix_Day3,Acidity,FruitWeight,LeafWeight,Yield
0,Variety1,1,25,5,10,0,15,21.2,20.0,33.333333,66.666667,111.92,2798.00
1,Variety1,2,24,4,8,0,13,22.4,23.6,30.769231,61.538462,78.17,1797.91
2,Variety1,1,24,4,10,0,14,21.8,25.0,28.571429,71.428571,90.55,2082.65
3,Variety1,1,16,8,4,0,12,21.2,23.2,66.666667,33.333333,83.97,1343.52
4,Variety1,1,22,6,8,0,14,20.4,23.4,42.857143,57.142857,120.95,2660.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Variety2,1,9,5,2,0,7,14.2,15.0,71.428571,28.571429,301.27,3615.24
106,Variety2,1,9,7,1,0,8,16.8,17.2,87.500000,12.500000,553.07,4977.63
107,Variety2,1,14,12,1,0,13,19.2,21.2,92.307692,7.692308,436.88,5679.44
108,Variety2,1,13,13,0,0,13,17.4,16.4,100.000000,0.000000,336.64,4376.32


If our data file type is other, we can use:
>pd.read_csv(filename)  
>pd.read_table(filename)  
>pd.read_excel(filename)  
>pd.read_sql(query, connection_object)  
>pd.read_json(json_string)  
>pd.read_html(url)  
>pd.read_clipboard()  

### Detecting outliers function

In [6]:
def detectingOutliers(df,outlierValue=1.5,extremeValue=3,txtName='outliersTest.txt'):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num = df.select_dtypes(include=numerics)
    columns=df_num.columns.values
    filew=open(txtName,'w')
    for column in columns:
        values=df[column].dropna().tolist()
        outliers=[]
        extremeOutliers=[]
        Q1=np.percentile(df[column].dropna(),25)
        Q3=np.percentile(df[column].dropna(),75)
        IQR=Q3-Q1
        for value in values:
            if value < (Q1-IQR*extremeValue) or value > (Q3+IQR*extremeValue):
                extremeOutliers.append(value)
            if value < (Q1-IQR*outlierValue) or value > (Q3+IQR*outlierValue):
                outliers.append(value)
        filew.write('Column {}:\n'.format(column))
        if len(outliers)==0:
            filew.write('\tNo outliers detected\n\n')
        elif len(extremeOutliers)==0:
            filew.write('\tNo extreme outliers detected\n')
            filew.write('\tOutliers at rows:\n')
            for value in set(outliers):
                index=df.index[df[column] == value].tolist()
                
                for ind in index:
                    filew.write('\t\t{}\t(value: {})\n'.format(ind+1,value))
            filew.write('\n')
        else:
            filew.write('\tExtreme outliers at rows:\n')
            for value in set(extremeOutliers):
                index=df.index[df[column] == value].tolist()
                for ind in index:
                    filew.write('\t\t{}\t(value: {})\n'.format(ind+1,value))
            filew.write('\tOutliers at rows:\n')
            for value in set(outliers):
                index=df.index[df[column] == value].tolist() 
                for ind in index:
                    filew.write('\t\t{}\t(value: {})\n'.format(ind+1,value))
            filew.write('\n')
detectingOutliers(data)

Note than predefined multiple of IQR (Interquartile range) is:  
>1.5 for outliers  
>3 for extreme outliers  

You can change this values with parameters outlierValue and extremeValue:

In [8]:
detectingOutliers(data,outlierValue=2,extremeValue=4)

Predefined txt output name is 'outliersTest.txt'; you can also assign the name you want for txt outuput with parameter txtName.

NOTE:  
Missing values input can be:
>'' (empty cell)  
>'#N/A'  
'#N/A N/A'  
'#NA'  
'-1.#IND'  
'-1.#QNAN'  
'-NaN'  
'-nan'  
'1.#IND'  
'1.#QNAN'  
'N/A'  
'NA'  
'NULL'  
'NaN'  
'n/a'  
'nan'  
'null'